In [1]:
!pip install opencv-python pillow torch torchvision diffusers transformers
!pip install rembg
!pip install onnxruntime
from google.colab import drive
drive.mount('/content/drive')
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install opencv-python matplotlib numpy Pillow torch torchvision
!wget -P /content/ "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth"
!pip install diffusers==0.16.1 transformers accelerate torch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-0q20yh9y
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-0q20yh9y
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
--2025-03-14 17:26:43--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.173.166.31, 18.173.166.51, 18.173.166.48, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.173.166.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘/content/sam_vit_b_01ec64.pth.1’

sam_vit_b_01ec64.pt 100%[=========

In [14]:
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
checkpoint_path = "/content/sam_vit_b_01ec64.pth"

def load_sam_model(model_type="vit_b", checkpoint_path=checkpoint_path):
    sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
    sam.to("cpu")
    return SamAutomaticMaskGenerator(sam)
mask_generator = load_sam_model()
print("Model loaded successfully!")




Model loaded successfully!


Pre-processing images

In [20]:
from PIL import Image

def composite_image(foreground_path, background_path, output_path, position=(0, 0)):
    bg = Image.open(background_path).convert("RGBA")
    fg = Image.open(foreground_path).convert("RGBA")
    bg.paste(fg, position, fg)
    bg.save(output_path)
    print(f"Composited image saved to {output_path}")



In [21]:
import os
import cv2
import numpy as np

def remove_white_bg(input_img_path, output_img_path,
                    lower_thresh=(200, 200, 200),
                    upper_thresh=(255, 255, 255)):
    bgr = cv2.imread(input_img_path)
    if bgr is None:
        print(f"Could not read: {input_img_path}")
        return
    mask = cv2.inRange(bgr, lower_thresh, upper_thresh)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)
    bgra = cv2.cvtColor(bgr, cv2.COLOR_BGR2BGRA)
    bgra[mask == 255, 3] = 0
    cv2.imwrite(output_img_path, bgra)
    print(f"✅ Processed and saved: {output_img_path}")

def batch_remove_white_bg(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for filename in os.listdir(input_folder):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            input_path = os.path.join(input_folder, filename)
            base_name = os.path.splitext(filename)[0]
            output_filename = f"{base_name}_no_bg.png"
            output_path = os.path.join(output_folder, output_filename)
            remove_white_bg(input_path, output_path)
if __name__ == "__main__":
    input_folder = "/content/drive/MyDrive/Collab dataset/471034853001-FLEP"
    output_folder = "/content/drive/MyDrive/Collab dataset/471034853001-FLEP/processed"
    batch_remove_white_bg(input_folder, output_folder)

    print("🎉 All images processed successfully!")


✅ Processed and saved: /content/drive/MyDrive/Collab dataset/471034853001-FLEP/processed/471034853001-0393_no_bg.png
✅ Processed and saved: /content/drive/MyDrive/Collab dataset/471034853001-FLEP/processed/471034853001-0395_no_bg.png
✅ Processed and saved: /content/drive/MyDrive/Collab dataset/471034853001-FLEP/processed/471034853001-0429_no_bg.png
✅ Processed and saved: /content/drive/MyDrive/Collab dataset/471034853001-FLEP/processed/471034853001-0396_no_bg.png
✅ Processed and saved: /content/drive/MyDrive/Collab dataset/471034853001-FLEP/processed/471034853001-0441_no_bg.png
✅ Processed and saved: /content/drive/MyDrive/Collab dataset/471034853001-FLEP/processed/471034853001-0434_no_bg.png
✅ Processed and saved: /content/drive/MyDrive/Collab dataset/471034853001-FLEP/processed/471034853001-0428_no_bg.png
🎉 All images processed successfully!


In [15]:
import os
from PIL import Image

def composite_carpet_on_interior(interior_image_path, carpet_image_path, output_path, position, scale=None):
    background = Image.open(interior_image_path).convert("RGBA")
    carpet = Image.open(carpet_image_path).convert("RGBA")
    if scale is not None:
        carpet = carpet.resize(scale, resample=Image.LANCZOS)
    background.paste(carpet, position, carpet)
    root, ext = os.path.splitext(output_path)
    if ext == "" or ext.lower() not in [".png", ".jpg", ".jpeg"]:
        output_path = root + ".png"

    background.save(output_path, format="PNG")
    print(f"Composite saved: {output_path}")

def batch_composite_carpet(interiors_folder, carpet_image_path, output_folder, position, scale=None):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(interiors_folder):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):
            interior_path = os.path.join(interiors_folder, filename)
            output_path = os.path.join(output_folder, "composite_" + filename)
            composite_carpet_on_interior(interior_path, carpet_image_path, output_path, position, scale)
if __name__ == "__main__":
    interiors_folder = "/content/drive/MyDrive/Generate_Interiors"
    carpet_image_path = "/content/drive/MyDrive/Collab dataset/471034853001-FLEP/Processed/471034853001-0428_no_bg.png"
    output_folder = "/content/drive/MyDrive/Finals_Interior"
    position = (400, 600)
    scale = (300, 150)
    batch_composite_carpet(interiors_folder, carpet_image_path, output_folder, position, scale)
    print("🎉 All composite images have been saved successfully!")


Composite saved: /content/drive/MyDrive/Finals_Interior/composite_interior_0_0.png
🎉 All composite images have been saved successfully!
